In [3]:
import requests as req
import os
import json
from pprint import pprint

import numpy as np
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000


In [5]:
#### Download Google Spreadsheets ####

# Make a temporary directory, and move into it
!mkdir temp
os.chdir("temp")
dest = os.getcwd()

# July Data Sheet
!curl "https://docs.google.com/spreadsheets/d/1viPOGYIk6RGu7YMoM3BHNVbkWaCZ0JFBOMSNncWvHYk/export?format=tsv" > july_data_upload.tsv
july_data_upload = pd.read_csv(dest+"/july_data_upload.tsv", sep="\t", index_col=[0])

# # Metadata to Upload
# !curl "https://docs.google.com/spreadsheets/d/1UkABgMlBIinJjITa6WepFAL-8VBkulS0LCbKojRXjVY/export?format=tsv" > current_metadata.tsv
# current_mdata = pd.read_csv(dest+"/current_metadata.tsv", sep="\t", index_col=[0])
# current_mdata = current_mdata.transpose()

!curl "https://docs.google.com/spreadsheets/d/1UkABgMlBIinJjITa6WepFAL-8VBkulS0LCbKojRXjVY/export?format=tsv" > current_metadata.tsv
current_mdata = pd.read_csv(dest+"/current_metadata.tsv", sep="\t", index_col=[0])
current_mdata = current_mdata.transpose()

!curl "https://docs.google.com/spreadsheets/d/1trzei7ETctQuI9kNg012MHieRVOLLAt1kbaKarT41fI/export?format=tsv" > fakemetabeta.tsv
#fakemetabeta = pd.read_csv(dest+"/fakemetabeta.tsv", sep="\t", index_col=[0])
#fakemetabeta = fakemetabeta.transpose()

# Delete temporary files
os.chdir("..")
!rm -r temp

#### Merge info from the Tracking sheet and Metadata sheet

old_id_col = "VIZZ - RW API (bulk upload)"
new_id_col = "API-ID (PERFECT DATASET)"
dl_from_src_col = "Download from Source"
dl_from_s3_col = "Download Data (S3)"

columns_to_xfer = [old_id_col, new_id_col, dl_from_src_col, dl_from_s3_col]
match_col = "Unique ID"

# HELPER FUNCTION

def append_columns(src_df, dst_df, columns_to_xfer, match_col):
    try:
        info = src_df.loc[dst_df[match_col], columns_to_xfer]
        # Have to set index or join returns no cols
        dst_df.set_index(match_col, inplace=True)
        dst_df[columns_to_xfer] = info
        dst_df.reset_index(inplace=True)
        return(dst_df)
    except:
        print("ERROR: There is an element in the metadata sheet that is not in the tracking sheet")
        return(None)
    
# current_mdata[old_id_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,old_id_col), axis=1)
# current_mdata[new_id_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,new_id_col), axis=1)

# current_mdata[dl_from_src_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,dl_from_src_col), axis=1)
# current_mdata[dl_from_s3_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,dl_from_s3_col), axis=1)

append_columns(july_data_upload, current_mdata, columns_to_xfer, match_col)

# Keep only those datasets with rw_ids already
valid_old_ids = pd.notnull(current_mdata[old_id_col])
valid_new_ids = pd.notnull(current_mdata[new_id_col])

def choose_new_id(df, valid_old_ids,old_id_col, valid_new_ids,new_id_col):
    assert(len(valid_old_ids)==len(valid_new_ids))
    final_ids = []
    for i in range(0, len(valid_new_ids)):
        if(valid_new_ids[i]):
            final_ids.append(df.iloc[i][new_id_col])
        elif(valid_old_ids[i]):
            final_ids.append(df.iloc[i][old_id_col])
        else:
            final_ids.append(None)
    return(final_ids)

current_mdata["final_ids"] = choose_new_id(current_mdata, valid_old_ids,old_id_col, valid_new_ids,new_id_col)
keep_matched_ids = pd.notnull(current_mdata["final_ids"])


matched_mdata = current_mdata.loc[keep_matched_ids]
matched_mdata.set_index("final_ids", inplace=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 87161    0 87161    0     0   210k      0 --:--:-- --:--:-- --:--:--  210k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  577k    0  577k    0     0  1903k      0 --:--:-- --:--:-- --:--:-- 1906k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100k    0  100k    0     0   545k      0 --:--:-- --:--:-- --:--:--  547k


In [6]:
matched_mdata

,Unique ID,Public Title,Technical Title,Subtitle,Source Organizations,Learn More Link,Function,Description,Cautions,Geographic Coverage,Data Type,Spatial Resolution,Date of Content,Frequency of Updates,Summary of Licence,Link to License,Citation,Published Language,Published Title (if not English),Download,Download from Source,Uploaded To,Layer Name 1,Layer Definition 1,Layer Name 2,Layer Definition 2,Layer Name 3,Layer Definition 3,Layer Name 4,Layer Definition 4,Original Data Name 1,Original Data Link 1,Original Data Name 2,Original Data Link 2,Original Data Name 3,Original Data Link 3,Original Data Name 4,Original Data Link 4,nan,VIZZ - RW API (bulk upload),API-ID (PERFECT DATASET),Download Data (S3)
final_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93ee443e-cb39-424a-9aa4-1d16af813418,bio.001,Endangered Species Sites,NaN,AZE,Alliance for Zero Extinction (AZE),http://www.biodiversitya-z.org/content/allianc...,Global list of sites containing 95% or more of...,Created by the Alliance for Zero Extinction (A...,None listed.,Global,Vector,NaN,2010,5 years,The AZE data and any derivatives may be used f...,https://www.arcgis.com/home/item.html?id=4ecca...,"Alliance for Zero Extinction. 2010. ""2010 AZE ...",English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93ee443e-cb39-424a-9aa4-1d16af813418,93ee443e-cb39-424a-9aa4-1d16af813418,NaN
4458eb12-8572-45d1-bf07-d5a3ee097021,bio.002,Biodiversity Hotspots,Biodiversity Hotspots Revisited 2011,CI/CEPF,Conservation International Foundation (CI)/Cri...,http://www.cepf.net/resources/hotspots/Pages/d...,Location and extent of the 35 biodiversity hot...,First defined in 1988 by scientist Norman Myer...,This layer only displays the land-based portio...,Global,Vector,NaN,Edited in 2016,No known updates,Creative Commons Attribution-ShareAlike 4.0 In...,http://www.cepf.net/resources/hotspots/Pages/d...,"Science and Knowledge, Conservation Internatio...",English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4458eb12-8572-45d1-bf07-d5a3ee097021,NaN,NaN
16df8ada-87cc-4907-adce-a98bc4e91856,bio.003,Marine Species Richness,Hotspots of Marine Biodiversity,EBD-CSIC/Phillip Island Nature Parks/Universit...,Estación Biológica de Doñana–Consejo Superior ...,http://advances.sciencemag.org/content/3/2/e16...,On the basis of the worldwide distribution (th...,To calculate the impact of climate change on m...,Fishing data are released with very low resolu...,Global (Marine Environment),Raster,0.5 degree,Varies,No known updates,Creative Commons Attribution-NonCommercial lic...,https://creativecommons.org/licenses/by-nc/4.0/,"Ramirez, Francisco, Isabel Afan, Lloyd S. Davi...",English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16df8ada-87cc-4907-adce-a98bc4e91856,NaN,NaN
3624554e-b240-4edb-9110-1f010642c3f3,bio.004,Coral Reef Locations,NaN,UNEP WCMC,United Nations Environment Programme World Con...,http://data.unep-wcmc.org/datasets/1,A record of global coral reef locations on a 5...,The UNEP WCMC compiled the Global Coral Reef m...,These data were compiled in 2001 and 2005 and ...,Global,Raster,500 m,"Data from 2001, 2005",No known updates,"No commercial use, no redistributing data",https://www.unep-wcmc.org/policies/general-dat...,"UNEP-WCMC, WorldFish Centre, WRI, and TNC. 201...",English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3624554e-b240-4edb-9110-1f010642c3f3,3624554e-b240-4edb-9110-1f010642c3f3,NaN
ad790c87-fe9e-4405-891d-de7c2ddfda79,bio.005,Coral Reef Bleaching Alerts,NaN,NOAA CRW,National Oceanic and Atmospheric Administratio...,https://coralreefwatch.noaa.gov/satellite/blea...,Displays the maximum level of coral bleaching ...,The Coral Reef Watch (CRW) program of the Nati...,These data layers do not contain any informati...,Global,Raster,5 km,2013-present,daily (13:30 EST),Public domain,https://wiki.creativecommons.org/wiki/Public_d...,NOAA

In [ ]:
list1 = [None, "a", None, "b"]
list2 = ["c", None, None, "d"]
df = pd.DataFrame({"l1":list1, "l2":list2})
choose_new_id(df, list1,"l1", list2,"l2")

In [ ]:
print(matched_mdata.shape)
print(july_data_upload.shape)

In [ ]:
sum(valid_new_ids)

In [ ]:
matched_mdata.head(1)

In [ ]:
july_data_upload.head(1)

In [ ]:
matched_mdata.columns

In [7]:
## FOR EACH DATASET IN BACKOFFICE THAT HAS METADATA, UPLOAD IT
from configparser import ConfigParser
config = ConfigParser()
config.read("../.env")
api_token = config.get("auth", "api_token")

auth_token = api_token # <Insert Auth Token Here>

# HELPER FUNCTIONS

def clean_nulls(val):
    try:
        if np.isnan(val):
            return(None)
        else:
            return(val)
    except:
        return(val)

def create_source_object(sources):
    if sources:
        source_object = []
        srcs = sources.split("/")
        for ix, src in enumerate(srcs):
            source_object.append({
                "source-name":src,
                "id":ix,
                "source-description":""
            })
        return source_object
    return None

processed1 = []


### THIS ADDS ALL DATASETS FOR WHICH WE HAVE METADATA in METADATA FOR UPLOAD ###

small_batch = ["5e69cfac-1f68-4864-a19a-3c1bdb180100"]
for rw_id in matched_mdata.index:
#for rw_id in small_batch:
    url = "https://api.resourcewatch.org/v1/dataset/"+str(rw_id)+"/metadata"
    print(url)
    # Everything from current_mdata
    metadata = matched_mdata.loc[rw_id]
    if len(metadata.shape) > 1:
        print(metadata)
        
    #print(metadata)
    row_payload = {
        "language": "en",
        
        "name": clean_nulls(metadata["Public Title"]),
        "description": clean_nulls(metadata["Description"]),
        "subtitle": clean_nulls(metadata["Subtitle"]),
        "functions": clean_nulls(metadata["Function"]),
        
        "application":"rw",
        "dataset":rw_id,
        
        "info": {
            
            # One of these a duplicate, test how shows up in front-end
            # or should rwId be dataset, above?
            "wri_rw_id": clean_nulls(metadata["Unique ID"]),
            "rwId": clean_nulls(metadata["Unique ID"]),
            
            "data_type": clean_nulls(metadata["Data Type"]),

            "name": clean_nulls(metadata["Public Title"]),
            "sources": create_source_object(clean_nulls(metadata["Source Organizations"])),
            
            "technical_title":clean_nulls(metadata["Technical Title"]),
            
            "functions": clean_nulls(metadata["Function"]),
            "cautions": clean_nulls(metadata["Cautions"]),
            
            "citation": clean_nulls(metadata["Citation"]),
            
            "license": clean_nulls(metadata["Summary of Licence"]),
            "license_link": clean_nulls(metadata["Link to License"]),
            
            "geographic_coverage": clean_nulls(metadata["Geographic Coverage"]),
            "spatial_resolution": clean_nulls(metadata["Spatial Resolution"]),
            
            "date_of_content": clean_nulls(metadata["Date of Content"]),
            "frequency_of_updates": clean_nulls(metadata["Frequency of Updates"]),
            
            "learn_more_link": clean_nulls(metadata["Learn More Link"]),
            "data_download_link": clean_nulls(metadata["Download Data (S3)"]),
            "data_download_original_link":clean_nulls(metadata["Download from Source"])
            
        }
    }

    headers = {
        'content-type': "application/json",
        'authorization': "Bearer " + auth_token,
    }

    try:
        processed1.append(rw_id)
        res = req.request("POST", url, data=json.dumps(row_payload), headers = headers)
        print(res)
        #print(res.text)
        if("already exists" in res.text):
            res = req.request("PATCH", url, data=json.dumps(row_payload), headers = headers)
            print(res)
            if("errors:" in res.text):
                print(res.text)
        elif("errors:" in res.text):
            print(res.text)
    except TypeError as e:
        print(e.args)
        print(metadata[["Unique ID", "Public Title"]])

https://api.resourcewatch.org/v1/dataset/93ee443e-cb39-424a-9aa4-1d16af813418/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/4458eb12-8572-45d1-bf07-d5a3ee097021/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/16df8ada-87cc-4907-adce-a98bc4e91856/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/3624554e-b240-4edb-9110-1f010642c3f3/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/ad790c87-fe9e-4405-891d-de7c2ddfda79/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/7d3465f8-5959-4531-aaf2-c9a8a03183b3/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/de452a4c-a55c-464d-9037-8c3e9fe48365/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1/dataset/3c82c421-8964-444e-86f2-df800174d8b9/metadata
<Response [400]>
<Response [200]>
https://api.resourcewatch.org/v1

In [8]:
#### Merge subtitles with Tracking sheet

#### UPLOADS TITLE, SUBTITLE, AND DOWNLOAD LINKS, if there is no METADATA IN METADATA FOR UPLOAD AVAILABEL ####

# Keep only those datasets from trakcing sheet with rw_ids already
tracking_valid_old_ids = pd.notnull(july_data_upload[old_id_col])
tracking_valid_new_ids = pd.notnull(july_data_upload[new_id_col])

july_data_upload["final_ids"] = choose_new_id(july_data_upload, tracking_valid_old_ids,old_id_col, tracking_valid_new_ids,new_id_col)

missed_ids = [rw_id for rw_id in july_data_upload["final_ids"].values if ((rw_id not in matched_mdata.index) and (rw_id != None))]

missed_data = july_data_upload.reset_index().set_index("final_ids")
missed_data = missed_data.loc[missed_ids]
missed_data


### THIS ADDS ALL DATASETS FOR WHICH WE HAVE ENTRIES IN TRACKING SHEET and NOTHING IN METADATA FOR UPLOAD###
print("True if below print empty list []")
print([ind for ind in missed_data.index if ind in matched_mdata.index])

processed2 = []

for rw_id in missed_data.index:
    url = "https://api.resourcewatch.org/v1/dataset/"+str(rw_id)+"/metadata"
    # Everything from current_mdata
    metadata = missed_data.loc[rw_id]
    print(metadata["WRI Unique ID"])
    print(metadata["Public Title"])
    print(url)
    #print(metadata)
    row_payload = {
        "language": "en",
        
        "name": clean_nulls(metadata["Public Title"]),
        "source": clean_nulls(metadata["Subtitle"]),
        
        "application":"rw",
        "dataset":rw_id,
        
        "info": {
            
            "wri_rw_id": clean_nulls(metadata["WRI Unique ID"]),

            "name": clean_nulls(metadata["Public Title"]),
            "technical_title":clean_nulls(metadata["Technical Title"]),

            "data_download_link": clean_nulls(metadata["Download Data (S3)"]), 
            "data_download_original_link": clean_nulls(metadata["Download from Source"])
            
        }
    }

    headers = {
        'content-type': "application/json",
        'authorization': "Bearer " + auth_token,
    }
    #print(row_payload)

    try:
        processed2.append(rw_id)
        res = req.request("POST", url, data=json.dumps(row_payload), headers = headers)
        if("already exists" in res.text):
            res = req.request("PATCH", url, data=json.dumps(row_payload), headers = headers)
            if("errors:" in res.text):
                print(res.text)
        elif("errors:" in res.text):
            print(res.text)
    except TypeError as e:
        print(e.args)
        print(metadata[["Unique ID", "Public Title"]])



True if below print empty list []
[]
bio.031
Social and Economic Dependence on Coral Reefs
https://api.resourcewatch.org/v1/dataset/894f43a8-ce8e-43a5-a4c7-fa80faa43d63/metadata
bio.033
Cold Water Corals
https://api.resourcewatch.org/v1/dataset/6b8442f5-4766-4444-94b4-d6676277fd80/metadata
bio.035
Coral Bleaching Frequency Prediction
https://api.resourcewatch.org/v1/dataset/1ef55baf-bbbe-480d-85e9-7132c742f196/metadata
https://wri-public-data.s3.amazonaws.com/resourcewatch/raster/cit_033_urban_built_up_areas/cit_033_urban_built_up_areas_merge.tif   
Human Settlements - Built Up Areas
https://api.resourcewatch.org/v1/dataset/7f2b61c1-d6b8-4601-8a86-5e5f9c125f4b/metadata
cit.032
Urban Population Projections
https://api.resourcewatch.org/v1/dataset/7d9c0d09-e833-4a74-811b-0af78da9c731/metadata
cli.003
Sea Surface Temperature Anomalies
https://api.resourcewatch.org/v1/dataset/c8040a7a-a40f-48bd-b003-625c33beff5e/metadata
cli.013
CH4 Concentrations
https://api.resourcewatch.org/v1/dataset/c

In [9]:
missed_data.to_csv("/Users/nathansuberi/Desktop/datasets_on_july_sheet_with_rw_id_no_metadata.csv")
missed_data

,WRI Unique ID,VIZZ - RW API (bulk upload),Slug - RW API (sanity check),API-ID (PERFECT DATASET),Public Title,Alternative Public Title,Technical Title,Subtitle,Theme_1,Theme_2,Theme_3,Planet Pulse,Water & Conflict,Problem Solving,Process these first,Multiple Layers needed to surface at same time?,Metadata Completed,Metadata on Backoffice,Distribution Restriction,Tags,Format,"Endpoint URL (Carto, GEE)",Download from Source,Download Data (S3),On WRI Platform,Dataset Processed for Upload,Data Upload Responsibility,Uploaded to S3,Server Location,Missing ISO Code,Dataset on Backoffice,Columns defined on Backoffice,Layer Definition,Layer Name/Description finalized,Widgets,Published on RW
final_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
894f43a8-ce8e-43a5-a4c7-fa80faa43d63,bio.031,894f43a8-ce8e-43a5-a4c7-fa80faa43d63,NaN,894f43a8-ce8e-43a5-a4c7-fa80faa43d63,Social and Economic Dependence on Coral Reefs,NaN,Social and Economic Dependence on Coral Reefs,WRI,Biodiversity,Commerce,Society,NaN,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://datasets.wri.org/dataset/reefs-at-risk-...,NaN,NaN,NaN,Liz,NaN,Carto,NaN,X,NaN,X,X,NaN,X
6b8442f5-4766-4444-94b4-d6676277fd80,bio.033,6b8442f5-4766-4444-94b4-d6676277fd80,NaN,6b8442f5-4766-4444-94b4-d6676277fd80,Cold Water Corals,NaN,Global DIstribution of Cold Water Corals,UNEP-WCMC,Biodiversity,NaN,NaN,NaN,NaN,Max/Laura write metadata,NaN,NaN,ready,NaN,X,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://data.unep-wcmc.org/datasets/3,https://wri-public-data.s3.amazonaws.com/resou...,NaN,NaN,Elise,X,Carto,NaN,X,NaN,X,X,NaN,X
1ef55baf-bbbe-480d-85e9-7132c742f196,bio.035,1ef55baf-bbbe-480d-85e9-7132c742f196,NaN,NaN,Coral Bleaching Frequency Prediction,NaN,Frequency of Future Coral Reef Bleaching Event...,WRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7f2b61c1-d6b8-4601-8a86-5e5f9c125f4b,https://wri-public-data.s3.amazonaws.com/resou...,7f2b61c1-d6b8-4601-8a86-5e5f9c125f4b,NaN,NaN,Human Settlements - Built Up Areas,NaN,Global Human Settlement - Built Up Grid,EU JRC,Cities,NaN,NaN,pulse context,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Raster,NaN,http://ghsl.jrc.ec.europa.eu/ghs_smod.php,NaN,NaN,NaN,Nathan,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7d9c0d09-e833-4a74-811b-0af78da9c731,cit.032,7d9c0d09-e833-4a74-811b-0af78da9c731,NaN,7d9c0d09-e833-4a74-811b-0af78da9c731,Urban Population Projections,NaN,Urban Population Percentage,No Metadata Yet,Cities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ready,NaN,X,NaN,CSV,https://resourcewatch.carto.com/u/wri-rw/datas...,https://esa.un.org/unpd/wup/CD-ROM/,https://wri-projects.s3.amazonaws.com/resource...,NaN,NaN,Emily,X,Carto,NaN,X,NaN,X,NaN,X,X
c8040a7a-a40f-48bd-b003-625c33beff5e,cli.003,c8040a7a-a40f-48bd-b003-625c33beff5e,Sea-Surface-Temperature-Anomalies-NOAA,NaN,Sea Surface Temperature Anomalies,NaN,Twice-weekly Sea Surface Temperature Anomalies,No Metadata Yet,Climate,NaN,NaN,RT,NaN,Vizz rasters - who makes layer?,NaN,NaN,ready,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,?,NaN,NaN,X
c9c9cb2f-9655-4f40-8736-9b407ee43514,cli.013,c9c9cb2f-9655-4f40-8736-9b407ee43514,Current-Methane-Concentration-CH4,NaN,CH4 Concentrations,NaN,"Current Methane Concentration, CH4",No Metadata Yet,Climate,NaN,NaN,RT,NaN,Laura/Max write metadata,NaN,NaN,Laura,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,X,NaN,NaN,NaN
ea67f436-473c-4977-bdf2-8aa0dabbaa6f,cli.021,ea67f436-473c-4977-bdf2-8aa0dabbaa6f,NaN,NaN,Average Snow Cover,NaN,Northern Hemisphere Average Snow Cover by Mont...,No Metadata Yet,Climate,NaN,NaN,RT,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,Raster,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN
d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a,cli.025,d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a,Sea-Level-Trends,NaN,Relative Sea Level Trends,NaN,Global Linear Relative Mean Sea Level (MSL) tr...,No Metadata Yet,Climate,Cities,NaN,explore,NaN,NaN,NaN,NaN,ready,N

In [ ]:
"9ea634db-53af-445e-a767-60ec9efc321e" in processed2

In [ ]:
#### Inspect metadata on backoffice

# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[atts["name"]] = {
        "rw_id":dset["id"],
        "upload_name":atts["name"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata_keys":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }    
    
# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')

def check_public_title(metadata):
    if len(metadata) > 0:
        mdata = metadata[0]
        if "attributes" in mdata:
            if "info" in mdata["attributes"]:
                if "name" in mdata["attributes"]["info"]:
                    return(mdata["attributes"]["info"]["name"])
        return(None)

# Grab public title, if it exists in metadata
current_datasets_on_api["public_title"] = current_datasets_on_api.apply(lambda row: check_public_title(row["metadata"]), axis=1)

current_datasets_on_api.set_index("rw_id", inplace=True)
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

In [ ]:
### THIS COVERS ALL DATASETS WHICH ARE ON THE BACKOFFICE but HAVE NO WRI_ID / RW_ID IN TRACKING SHEET ###
### Occasionally this is because the data has been moved to after launch

investigate_mdata = current_datasets_on_api[["upload_name", "public_title", "metadata"]]

missed_ids = [rw_id for rw_id in investigate_mdata.index if ((rw_id not in processed1) & (rw_id not in processed2))]

investigate_mdata = investigate_mdata.loc[missed_ids]

investigate_mdata

Many of these are datasets for which the Unique ID changed

soc.003 Distribution of Infant Mortality
soc.016 Conflict and Protest Events in African...
dis_007 Landslide Susceptibility Map
bio.035 Coral Bleaching Frequency Prediction
dis.001 Earthquakes Over the Past 30 days
Foo_046a Food Footprint in Protein
wat.033 Agriculture Water Demand and Depletion
soc.062 Internal Displacement
soc.061 Rural Poverty
soc.042 Percentage of Urban Population with Ac
soc.020 GINI Index
soc.008 Gross Domestic Product Per Capita (PPP
soc.006 Multidimensional Poverty Index
soc.004 Human Development Index
soc.002 Gender Development Index
foo.002 GLDAS Land Water Content from NOAH Lan..
com.028 Effect of Agricultural Policies on Com...
cit.029 Municipal Waste

In [ ]:
# DANGER Bug - able to update metadata for a dataset that no longer exists on the API
#test upload cit.029:
#    broken, old id: 8f14a33e-5a61-47af-b26e-c1fc036932a5
#    working, new id: 00abb46f-34e2-4bf7-be30-1fb0b1de022f
    
url1="https://api.resourcewatch.org/v1/dataset/8f14a33e-5a61-47af-b26e-c1fc036932a5/metadata"    
url2="https://api.resourcewatch.org/v1/dataset/10337db6-8321-445e-a60b-28fc1e114f29/metadata"

res1a = req.request("POST", url1, data=json.dumps(row_payload), headers = headers)
if("already exists" in res1a.text):
    res1b = req.request("PATCH", url1, data=json.dumps(row_payload), headers = headers)
        
res2a = req.request("POST", url2, data=json.dumps(row_payload), headers = headers)
if("already exists" in res2a.text):
    res2b = req.request("PATCH", url1, data=json.dumps(row_payload), headers = headers)
    
print(res1b.text)